In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/setuptools-74.1.2-py3-none-any.whl
/kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/autogluon.features-1.1.1-py3-none-any.whl
/kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/kiwisolver-1.4.7-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
/kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/tzdata-2024.1-py2.py3-none-any.whl
/kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/botocore-1.35.14-py3-none-any.whl
/kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/pillow-10.4.0-cp310-cp310-manylinux_2_28_x86_64.whl
/kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/urllib3-2.2.2-py3-none-any.whl
/kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/jmespath-1.0.1-py3-none-any.whl
/kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/s3transfer-0.10.2-py3-none-any.

In [2]:
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl
!pip install --no-index --find-links /kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular autogluon.tabular

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=d471cb6e4334319c6dd6334c9677431539cdb9fa0e61fb035be48a0d91dcc2a2
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl
Looking in links: /kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular
Processing /kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/autogluon.tabular-1.1.1-py3-none-any.whl
Processing /kaggle/input/autogluon-1-1-1-tabular/autogluon_tabular/scip

In [3]:
from autogluon.tabular import TabularPredictor
from lifelines import KaplanMeierFitter, CoxPHFitter, NelsonAalenFitter
from metric import score
from scipy.stats import rankdata 
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb
from lightgbm import LGBMRegressor,LGBMClassifier
import lightgbm as lgb
from catboost import CatBoostRegressor, CatBoostClassifier
import catboost as cb
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from pathlib import Path
import optuna
import polars as pl

In [4]:
def stratified_c_index(df, predictions):
    y_true = df[["ID", "efs", "efs_time","race_group"]].copy()
    y_pred = df[["ID"]].copy()
    y_pred["prediction"] = predictions
    m = score(y_true.copy(), y_pred.copy(),"ID")
    return m

In [5]:
class settings:

    train_link = "/kaggle/input/equity-post-HCT-survival-predictions/train.csv"
    test_link =  "/kaggle/input/equity-post-HCT-survival-predictions/test.csv"
    sub_link = "/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv"
    
    seed = 42
    target = 'target'
    n_splits = 10
    col_ignore = ['efs_time','efs','target','ID']

    batch_size = 32768
    

In [17]:
class SurvModel:

    def __init__(self, model, n_splits = settings.n_splits, random_state = settings.seed, estimator = 'na'):
        """
        Initialise the SurvModel class

        Parameters:
        - model: A model instance
        - n_splits: Number of folds for k-fold cross validation
        - random_state: for reproducability 
        """

        self.model = model
        self.n_splits = n_splits
        self.random_state = random_state
        self.estimator = estimator
        self.oof = None
        self.pred = None
        self.scores = []
        self.cats = []

    def preprocessing(self, train, test):
        """
        Proprocessing the test and train data

        Parameters
        - train: training dataw
        - test: testing data
        """
        # For now for testing lets use KM estmator
        # Fitted a survival curve to the training data to predict off
        #train['efs_time'] = np.log1p(train['efs_time'])
        if self.estimator == 'km':
            kmf = KaplanMeierFitter()
            kmf.fit(durations = train['efs_time'], event_observed = train['efs'])
            train[settings.target] = kmf.survival_function_at_times(train['efs_time']).values
            train[settings.target] = np.log(train[settings.target])
            print("#"*25,"KM Estmator Fitted", "#"*25)

        elif self.estimator == 'na':
            naf = NelsonAalenFitter()
            naf.fit(durations = train['efs_time'], event_observed = train['efs'])
            train[settings.target] = -naf.cumulative_hazard_at_times(train['efs_time']).values
            print("#"*25,"Nelson-Aalen Estmator Fitted", "#"*25)

        elif self.estimator == 'chris':
                time_col = 'efs_time'
                event_col = 'efs'
                train[settings.target] = train[time_col].values
                mx = train.loc[train[event_col]==1, time_col].max()
                mn = train.loc[train[event_col]==0, time_col].min()
                train.loc[train[event_col]==0, settings.target] = train.loc[train[event_col]==0, settings.target] + mx - mn
                train[settings.target] = train[settings.target].rank()
                train.loc[train[event_col]==0, settings.target] += 1*len(train)
                train[settings.target] = train[settings.target] / train[settings.target].max()
                train[settings.target] = np.log(train[settings.target])
                train[settings.target] -= train[settings.target].mean()
                train[settings.target] *= -1.0

        else:
            train[settings.target] = train.efs_time.copy()
            train.loc[train.efs==0,settings.target]*=-1
            print("#"*25,"CPH Fitted", "#"*25)

        features = [col for col in train.columns if not col in settings.col_ignore]
        
        # Combined test and train to label encode
        combined = pd.concat([train,test], axis = 0, ignore_index = True)

        for col in features:
            if combined[col].dtype == "object":
                self.cats.append(col)
                combined[col] = combined[col].fillna("NAN")
                combined[col],_ = combined[col].factorize()
                combined[col] -= combined[col].min()
                combined[col] = combined[col].astype("int32")
                combined[col] = combined[col].astype("category")

            else:
                if combined[col].dtype=="float64":
                    combined[col] = combined[col].astype("float32")
                if combined[col].dtype=="int64":
                    combined[col] = combined[col].astype("int32")
        train = combined.iloc[:len(train)].copy()
        test = combined.iloc[len(train):].reset_index(drop=True).copy().drop([settings.target,'efs','efs_time'], axis = 1)
        return train, test

    def fit_predict(self, train, test, catboost = False, autogluon = False):
        """
        Fit the model using k-fold cross validation and collect oof predictions.

        Note: the train and test data include ID, efs and efs_time as we need it for the metric.
              We need to remove them for training but add them back in for metric collection.
        
        Parameters:
        - Train: the training data
        - Test: the test data from kaggle

        Returns:
        - OOF predictions
        _ Preditions for the test data
        """
        # Collect features

        features = [col for col in train.columns if not col in settings.col_ignore]
        print("#"*25,"Features","#"*25)
        print(f"There are {len(features)} features.")
        print(features)

        # Initialise k-fold cross validation
        print("#"*25,"Initialise Cross-Validation","#"*25)

        kf = KFold(n_splits = self.n_splits, shuffle=True, random_state=self.random_state)

        # Predictions for OOF and test
        self.oof = np.zeros(len(train))
        self.preds = np.zeros(len(test))

        # Cross-validation loop
        for fold, (train_idx, test_idx) in enumerate(kf.split(train)):
            
            print("#"*25,f"Fold {fold+1}","#"*25)
            # Split data
            x_train, x_val = train.loc[train_idx, features].copy(), train.loc[test_idx, features].copy()
            y_train, y_val = train.loc[train_idx, settings.target].copy(), train.loc[test_idx, settings.target].copy()
            x_test = test[features]

            x_score = train.loc[test_idx,["ID", "efs", "efs_time","race_group"]].copy()
            
            # Train on training set
            if catboost:
                self.model.fit(x_train, y_train, self.cats, verbose = 0, eval_set=[(x_val,y_val)])
            elif autogluon:
                x_train[settings.target] = y_train
                predictor = TabularPredictor(label=settings.target, verbosity =0,  path = "AutogluonModels/CV").fit(x_train, time_limit=60*5)
                
            else:
                self.model.fit(x_train, y_train)

            # Predict on validation set
            if autogluon:
                val_preds = predictor.predict(x_val)
            else:
                val_preds = self.model.predict(x_val)
            self.oof[test_idx] = val_preds

            # Calculate score
            m = stratified_c_index(x_score, val_preds)
            self.scores.append(m)
            print(f"Fold {fold+1}: Stratified C-index: {m}")
            if autogluon:
                self.preds += predictor.predict(x_test)/self.n_splits
            else:
                self.preds += self.model.predict(x_test)/self.n_splits
            
        train_score = train[["ID", "efs", "efs_time","race_group"]].copy()
        print("#"*25,"Cross-Validation Completed","#"*25)
        print(f"Average Stratified C-Index: {np.mean(self.scores)}")
        print(f"Overall Stratified C-Index: {stratified_c_index(train_score, self.oof)}")
        return self.oof, self.preds, np.mean(self.scores)

    def save_predictions(self, filename,train, test):
        """
        Save predictions in a csv.

        Parameters:
        - filename: model_name
        - train: train data for ID
        - test: test data for ID
        """
        directory_train = Path("/kaggle/working/train")
        directory_test = Path("/kaggle/working/test")

        directory_train.mkdir(exist_ok = True)
        print(f"Directory '{directory_train}' created successfully")
        directory_test.mkdir(exist_ok = True)
        print(f"Directory '{directory_test}' created successfully")
            
        
        # oof train preds
        train_csv = train.copy()
        train_csv["prediction"] = self.oof

        train_csv.to_csv("train/"+filename+"_train.csv", index = False)

        # test
        test_csv = test.copy()
        test_csv["prediction"] = self.preds

        test_csv.to_csv("test/"+filename+"_test.csv", index = False)

        print(f"Out of fold predictions and test predictions have been saved")
        

In [7]:
class Ensembler:
    def __init__(self, train_folder, test_folder):
        """
        Initialse class with the train and test folders.

        Parameters:
        - train_folder: Path to the train folder containing OOF predictions
        - test_folder: Path to the predictions
        """

        self.train_folder = Path(train_folder)
        self.test_folder = Path(test_folder)
        self.scores = []

    def load_files(self, folder_path):
        """
        Load the train and test files.

        Returns:
        - train: consolidated predictions for each dataset
        - test: consolidated predictions for each dataset
        """
        predictions = {}

        # loop through the csv files
        counter = 0
        for idx, file in enumerate(sorted(folder_path.glob("*.csv"))):
            try:
                df = pd.read_csv(file)
                if "prediction" in df.columns:
                    counter +=1
                    predictions[f"prediction_{counter}"] = df["prediction"].reset_index(drop=True)
                    print(f"Loaded 'prediction' column from {file.name}")
                else:
                    print(f"'prediction' column not found in {file.name}")
            except Exception as e:
                print(f"Error loading {file.name}: {e}")
        if predictions:
            predictions = pd.DataFrame(predictions)
            predictions["ID"] = df['ID']
            return predictions
        else:
            print("No predictions found.")
            return pd.DataFrame()

    def concatenate_train_test(self, train):
        """
        Concatenate train and test folder data into another train and test set.

        Parameters:
        - train: we need this to get the target variable in the training set

        Returns
        - train: Another training set based on the oof predictions.
        - test: Another test set based on preds.
        """
        print(f"Loading predictions from the train folder {self.train_folder}")
        train_predictions = self.load_files(self.train_folder)
        train_predictions[settings.target] = train[settings.target].reset_index(drop=True)

        print(f"Loading predictions from the test folder {self.test_folder}")
        test_predictions = self.load_files(self.test_folder)

        return train_predictions, test_predictions

    def ranking(self, train, test, original_train):
        
        # Collect features

        features = [col for col in train.columns if not col in settings.col_ignore]
        print("#"*25,"Features","#"*25)
        print(f"There are {len(features)} features.")
        print(features)

        # Initialise k-fold cross validation
        print("#"*25,"Initialise Cross-Validation","#"*25)

        kf = KFold(n_splits = settings.n_splits, shuffle=True, random_state=settings.seed)

        # Cross-validation loop
        for fold, (train_idx, test_idx) in enumerate(kf.split(train)):
            
            print("#"*25,f"Fold {fold+1}","#"*25)
            # Split data
            x_train, x_val = train.loc[train_idx, features].copy(), train.loc[test_idx, features].copy()
            y_train, y_val = train.loc[train_idx, settings.target].copy(), train.loc[test_idx, settings.target].copy()
            x_test = test[features]

            x_score = original_train.loc[test_idx,["ID", "efs", "efs_time","race_group"]].copy()

            val_preds = np.zeros(len(x_val))
            
            # Predict on validation set
            for col in x_val:
                if col not in settings.col_ignore:
                    val_preds += rankdata(x_val[col])
                    


            # Calculate score
            m = stratified_c_index(x_score, val_preds)
            self.scores.append(m)
            print(f"Fold {fold+1}: Stratified C-index: {m}")
            
        overall_pred = np.zeros(len(original_train))
        for col in train:
            if col not in settings.col_ignore:
                overall_pred += rankdata(train[col])

        overall_score = original_train[["ID", "efs", "efs_time","race_group"]].copy()
            
        print("#"*25,"Cross-Validation Completed","#"*25)
        print(f"Average Stratified C-Index: {np.mean(self.scores)}")
        print(f"Overall Stratified C-Index: {stratified_c_index(overall_score, overall_pred)}")

        

        
        
        

In [8]:
class FE:
    def __init__(self, batch_size):
        self.batch_size = batch_size
        # self.scaler = StandardScaler()

    def load_data(self, path):
        return pl.read_csv(path, batch_size=self.batch_size)

    def recalculate_hla_sums(self, df):     
        df = df.with_columns(
            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) + 
             pl.col("hla_match_drb1_high").fill_null(0)).alias("hla_nmdp_6"),
            
            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) + 
             pl.col("hla_match_drb1_low").fill_null(0)).alias("hla_low_res_6"),
            
            (pl.col("hla_match_a_high").fill_null(0) + pl.col("hla_match_b_high").fill_null(0) + 
             pl.col("hla_match_drb1_high").fill_null(0)).alias("hla_high_res_6"),
            
            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) + 
             pl.col("hla_match_c_low").fill_null(0) + pl.col("hla_match_drb1_low").fill_null(0)
            ).alias("hla_low_res_8"),
            
            (pl.col("hla_match_a_high").fill_null(0) + pl.col("hla_match_b_high").fill_null(0) + 
             pl.col("hla_match_c_high").fill_null(0) + pl.col("hla_match_drb1_high").fill_null(0)
            ).alias("hla_high_res_8"),
            
            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) + 
             pl.col("hla_match_c_low").fill_null(0) + pl.col("hla_match_drb1_low").fill_null(0) +
             pl.col("hla_match_dqb1_low").fill_null(0)).alias("hla_low_res_10"),
            
            (pl.col("hla_match_a_high").fill_null(0) + pl.col("hla_match_b_high").fill_null(0) + 
             pl.col("hla_match_c_high").fill_null(0) + pl.col("hla_match_drb1_high").fill_null(0) +
             pl.col("hla_match_dqb1_high").fill_null(0)).alias("hla_high_res_10"),
        )
        return df

    def cast_datatypes(self, df):
        num_cols = [
            'hla_high_res_8', 'hla_low_res_8', 'hla_high_res_6',
            'hla_low_res_6', 'hla_high_res_10', 'hla_low_res_10',
            'hla_match_dqb1_high', 'hla_match_dqb1_low',
            'hla_match_drb1_high', 'hla_match_drb1_low',
            'hla_nmdp_6', 'year_hct', 'hla_match_a_high',
            'hla_match_a_low', 'hla_match_b_high', 'hla_match_b_low',
            'hla_match_c_high', 'hla_match_c_low', 'donor_age',
            'age_at_hct', 'comorbidity_score', 'karnofsky_score',
            'efs', 'efs_time'
        ]

        for col in df.columns:
            if col in num_cols:
                df = df.with_columns(pl.col(col).fill_null(-1).cast(pl.Float32))  

            else:
                df = df.with_columns(pl.col(col).fill_null('Unknown').cast(pl.String))  

        return df.with_columns(pl.col('ID').cast(pl.Int32))

    def info(self, df):     
        print(f'\nShape of dataframe: {df.shape}')    
        mem = df.memory_usage().sum() / 1024**2
        print('Memory usage: {:.2f} MB\n'.format(mem))
        display(df.head())

    def apply_fe(self, path):
        df = self.load_data(path)
        df = self.recalculate_hla_sums(df)
        df = self.cast_datatypes(df)
        df = df.to_pandas()
        #self.info(df)
        
        cat_cols = [col for col in df.columns if df[col].dtype == pl.String]
        return df, cat_cols

# Model Training

In [9]:
estimators = ['km','na','cox','chris']

In [10]:
lgb_params = {
        'objective': 'regression',
        'min_child_samples': 20,
        'num_iterations': 6000,
        'learning_rate': 0.01,
        'extra_trees': True,
        'reg_lambda': 3.0,
        'reg_alpha': 0.1,
        'num_leaves': 64,
        'metric': 'rmse',
        'max_depth': 10,
        'device': 'cpu',
        'max_bin': 255,
        'verbose': -1,
        'seed': 42
    }

for estimator in estimators:
    if estimator == 'cox':
        pass
    else:

        model = LGBMRegressor(**lgb_params
        )
        
        fe = FE(settings.batch_size)
        train, cat_cols = fe.apply_fe(settings.train_link)
        test, _ = fe.apply_fe(settings.test_link)
        pipeline = SurvModel(model, estimator = estimator)
        df_train, df_test = pipeline.preprocessing(train, test)
        oof_lgb, preds_lgb,_ = pipeline.fit_predict(df_train,df_test)
        pipeline.save_predictions("LightGBM_"+estimator, df_train, df_test)

######################### KM Estmator Fitted #########################
######################### Features #########################
There are 57 features.
['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_d

In [11]:
for estimator in estimators:
    if estimator == 'cox':
        model = XGBRegressor(
            device="cuda",
            max_depth=3,  
            colsample_bytree=0.5,  
            subsample=0.8,  
            n_estimators=2000,  
            learning_rate=0.02,  
            enable_categorical = True,
            objective = "survival:cox",
            eval_metric = 'cox-nloglik'
        )
        
    else:
        
        model = XGBRegressor(
            device="cuda",
            max_depth=3,  
            colsample_bytree=0.5,  
            subsample=0.8,  
            n_estimators=2000,  
            learning_rate=0.02,  
            enable_categorical=True,
            verbose=0
        )
    
    fe = FE(settings.batch_size)
    train, cat_cols = fe.apply_fe(settings.train_link)
    test, _ = fe.apply_fe(settings.test_link)
    pipeline = SurvModel(model, estimator = estimator)
    df_train, df_test = pipeline.preprocessing(train, test)
    oof_xgb, preds_xgb,_ = pipeline.fit_predict(df_train,df_test)
    pipeline.save_predictions("XGBoost_"+estimator, df_train, df_test)

######################### KM Estmator Fitted #########################
######################### Features #########################
There are 57 features.
['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_d

In [12]:
for estimator in estimators:
    if estimator == 'cox':
        model = CatBoostRegressor(loss_function = 'Cox', grow_policy='Lossguide',random_state = settings.seed)
    else:
        model = CatBoostRegressor(task_type = "GPU", grow_policy = 'Lossguide',random_state = settings.seed)

    fe = FE(settings.batch_size)
    train, cat_cols = fe.apply_fe(settings.train_link)
    test, _ = fe.apply_fe(settings.test_link)
    pipeline = SurvModel(model, estimator = estimator)
    df_train, df_test = pipeline.preprocessing(train, test)
    oof_cat, preds_cat,_ = pipeline.fit_predict(df_train,df_test, True)
    pipeline.save_predictions("Catboost_"+estimator, df_train, df_test)

######################### KM Estmator Fitted #########################
######################### Features #########################
There are 57 features.
['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_d

In [ ]:
for estimator in estimators:
    if estimator == 'cox':
        pass
    else:
        model = None

        fe = FE(settings.batch_size)
        train, cat_cols = fe.apply_fe(settings.train_link)
        test, _ = fe.apply_fe(settings.test_link)
        pipeline = SurvModel(model, estimator = estimator)
        df_train, df_test = pipeline.preprocessing(train, test)
        oof_auto, preds_auto,_ = pipeline.fit_predict(df_train,df_test, False,True)
        pipeline.save_predictions("Autogluon_"+estimator, df_train, df_test)

######################### KM Estmator Fitted #########################
######################### Features #########################
There are 57 features.
['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_d

# Ensembling

In [14]:
esm = Ensembler("/kaggle/working/train", "/kaggle/working/test")
final_train, final_test = esm.concatenate_train_test(df_train)
esm.ranking(final_train,final_test,df_train)

Loading predictions from the train folder /kaggle/working/train
Loaded 'prediction' column from Autogluon_chris_train.csv
Loaded 'prediction' column from Autogluon_km_train.csv
Loaded 'prediction' column from Autogluon_na_train.csv
Loaded 'prediction' column from Catboost_chris_train.csv
Loaded 'prediction' column from Catboost_cox_train.csv
Loaded 'prediction' column from Catboost_km_train.csv
Loaded 'prediction' column from Catboost_na_train.csv
Loaded 'prediction' column from LightGBM_chris_train.csv
Loaded 'prediction' column from LightGBM_km_train.csv
Loaded 'prediction' column from LightGBM_na_train.csv
Loaded 'prediction' column from XGBoost_chris_train.csv
Loaded 'prediction' column from XGBoost_cox_train.csv
Loaded 'prediction' column from XGBoost_km_train.csv
Loaded 'prediction' column from XGBoost_na_train.csv
Loading predictions from the test folder /kaggle/working/test
Loaded 'prediction' column from Autogluon_chris_test.csv
Loaded 'prediction' column from Autogluon_km_tes

# Submission

In [15]:
final_pred = np.zeros(len(final_test))

for col in final_test.columns:
    if col not in settings.col_ignore:
        final_pred += rankdata(final_test[col])


final_pred

array([28., 42., 14.])

In [16]:
submission = pd.read_csv(settings.sub_link)
submission['prediction'] = final_pred

submission.to_csv("submission.csv", index = False)
submission

,ID,prediction
0,28800,28.0
1,28801,42.0
2,28802,14.0
